In [ ]:
from google.colab import drive
import os
import zipfile

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
base_path = '/content/drive/My Drive/sec'
unzip_folder = 'unzipped'
zip_files = ['2010q1.zip', '2010q2.zip', '2010q3.zip', '2010q4.zip']

In [ ]:
def unzip_files(zip_files, base_path):
    for zip_file in zip_files:
        zip_path = os.path.join(base_path, zip_file)
        extract_dir = os.path.join(base_path, unzip_folder, f"{zip_file.replace('.zip', '')}")

        with zipfile.ZipFile(zip_path, 'r') as z:
            z.extractall(extract_dir)

unzip_files(zip_files, base_path)

I'm going to use Spark to merge the data for the 4 quarters. Using Pandas is fine as well but will probably cause memory issues.

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SEC Financial Data Analysis") \
    .getOrCreate()

def load_dataframes(zip_files, base_path):
    def load_zip_data(zip_file):
        extract_dir = os.path.join(base_path, unzip_folder, f"{zip_file.replace('.zip', '')}")

        sub_df = spark.read.csv(os.path.join(extract_dir, 'sub.txt'), sep='\t', header=True, inferSchema=True)
        num_df = spark.read.csv(os.path.join(extract_dir, 'num.txt'), sep='\t', header=True, inferSchema=True)
        tag_df = spark.read.csv(os.path.join(extract_dir, 'tag.txt'), sep='\t', header=True, inferSchema=True)
        pre_df = spark.read.csv(os.path.join(extract_dir, 'pre.txt'), sep='\t', header=True, inferSchema=True)

        return sub_df, num_df, tag_df, pre_df

    combined_sub, combined_num, combined_tag, combined_pre = load_zip_data(zip_files[0])

    for zip_file in zip_files[1:]:
        sub_df, num_df, tag_df, pre_df = load_zip_data(zip_file)
        combined_sub = combined_sub.union(sub_df)
        combined_num = combined_num.union(num_df)
        combined_tag = combined_tag.union(tag_df)
        combined_pre = combined_pre.union(pre_df)

    return combined_sub, combined_num, combined_tag, combined_pre


combined_sub, combined_num, combined_tag, combined_pre = load_dataframes(zip_files, base_path)

print(f"SUB: {combined_sub.count()} rows")
print(f"NUM: {combined_num.count()} rows")
print(f"TAG: {combined_tag.count()} rows")
print(f"PRE: {combined_pre.count()} rows")


SUB: 3904 rows
NUM: 1266146 rows
TAG: 66071 rows
PRE: 611781 rows


I'm focussing on just 16 companies to save myself time but the code is generic enough to handle the entire dataset.

In [ ]:
ciks = ['794367', '60667', '1166126', '104169', '78003', '1076405', '1403161', '200406', '72971', '1001039', '108772', '4962', '831001', '19617', '12927', '1403161', '1166691']

I'm doing the preprocessing in Spark and only transferring over relevant data to Neo4J, i.e. Neo4J is only being used for analytics. But if anyone of the groups is loading raw data into Neo4J and performing processing there, that's fine as well. Also, I'm focussing on just 5 attributes from the dataset which should suffice to provide relevant answers to some of the questions.
There are two types of nodes being generated: 1) Company, and 2) Financial instrument. Each company has a bunch of financial instruments with a timestamp which is going to help with looking at data over time.

In [ ]:
from pyspark.sql import functions as F

adsh_values = combined_sub.filter(combined_sub.cik.isin(ciks)).select("adsh", "cik").distinct()

financial_statements_num = (
    combined_num.join(adsh_values, on="adsh", how="inner")
)

financial_items_of_interest = [
    "SalesRevenueNet",
    "NetIncomeLoss",
    "Assets",
    "LiabilitiesCurrent",
    "DebtCurrent"
]

filtered_financial_statements = (
    financial_statements_num
    .filter(financial_statements_num.tag.isin(financial_items_of_interest))
    .join(combined_sub.select("cik", "name", "countryba", "stprba", "cityba"), on="cik", how="inner")
)

filtered_financial_statements = filtered_financial_statements.withColumn("value", filtered_financial_statements["value"].cast("double"))

filtered_financial_data = filtered_financial_statements.collect()

company_financials = {}

for row in filtered_financial_data:
    if row['value'] is not None:
        name = row['name'].replace("'", r'\'')
        country = row['countryba'].replace("'", r'\'')
        state = row['stprba'].replace("'", r'\'')
        city = row['cityba'].replace("'", r'\'')
        target_cik = row['cik']

        if target_cik not in company_financials:
            company_financials[target_cik] = {'name': name, 'country': country, 'state': state,  'city': city, 'instruments': []}

        company_financials[target_cik]['instruments'].append({
            'type': row['tag'],
            'value': row['value'],
            'date': row['ddate']
        })

cypher_queries = []

for target_cik, data in company_financials.items():
    name = data['name']
    country = data['country']
    state = data['state']
    city = data['city']
    cypher_query = f"MERGE (c:Company {{name: '{name}', country: '{country}', state: '{state}', city: '{city}', cik: '{target_cik}'}})"

    for i, instrument in enumerate(data['instruments']):
        cypher_query += (
            f"\nCREATE (f{i}:FinancialInstrument {{type: '{instrument['type']}', value: {instrument['value']}, date: date('{instrument['date']}')}}) "
            f"CREATE (c)-[:HAS_FINANCIAL_INSTRUMENT]->(f{i})"
        )

    cypher_queries.append(cypher_query)

print(cypher_queries)


["MERGE (c:Company {name: 'LOWES COMPANIES INC', country: 'US', state: 'NC', city: 'MOORESVILLE', cik: '60667'})\nCREATE (f0:FinancialInstrument {type: 'Assets', value: 32625000000.0, date: date('20090131')}) CREATE (c)-[:HAS_FINANCIAL_INSTRUMENT]->(f0)\nCREATE (f1:FinancialInstrument {type: 'Assets', value: 32625000000.0, date: date('20090131')}) CREATE (c)-[:HAS_FINANCIAL_INSTRUMENT]->(f1)\nCREATE (f2:FinancialInstrument {type: 'Assets', value: 32625000000.0, date: date('20090131')}) CREATE (c)-[:HAS_FINANCIAL_INSTRUMENT]->(f2)\nCREATE (f3:FinancialInstrument {type: 'Assets', value: 32625000000.0, date: date('20090131')}) CREATE (c)-[:HAS_FINANCIAL_INSTRUMENT]->(f3)\nCREATE (f4:FinancialInstrument {type: 'Assets', value: 33005000000.0, date: date('20100131')}) CREATE (c)-[:HAS_FINANCIAL_INSTRUMENT]->(f4)\nCREATE (f5:FinancialInstrument {type: 'Assets', value: 33005000000.0, date: date('20100131')}) CREATE (c)-[:HAS_FINANCIAL_INSTRUMENT]->(f5)\nCREATE (f6:FinancialInstrument {type: 'A

In [ ]:
!pip install neo4j yfiles_jupyter_graphs_for_neo4j pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.4 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.9
    Uninstalling widgetsnbextension-3.6.9:
      Successfully uninstalled widgetsnbextension-3.6.9
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [ ]:
from neo4j import GraphDatabase
driver = GraphDatabase.driver("neo4j+s://64ba9f5a.databases.neo4j.io:7687", auth=("neo4j", "Y7u178tcfqMFXavHHsFa5DOus9kinNSrRhfDxaP9yss"))

I'm doing a streaming insertion but loading directly from the Google Drive in the form of CSV might be more efficient.

In [ ]:
def insert_data(tx):
    for query in cypher_queries:
      tx.run(query)

with driver.session() as session:
    session.execute_write(insert_data)

Visualizing 3 nodes to get a feel for the graph.

In [ ]:
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget

g = Neo4jGraphWidget(driver)

def show_graph(driver):
    query = """
    MATCH (c:Company)-[r:HAS_FINANCIAL_INSTRUMENT]->(f:FinancialInstrument)
    WITH DISTINCT c, r, f
    RETURN c, COLLECT(r) AS relationships, COLLECT(f) AS instruments
    LIMIT 3
    """
    g.show_cypher(query)

show_graph(driver)


GraphWidget(layout=Layout(height='800px', width='100%'))

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

Q1: Analyze financial statements for companies. Financial Statement Comparison allows
analysts and investors to compare financial statements of different companies, aiding in
benchmarking and performance evaluation.  

---

Given a company CIK provide its financial statement. For revenue, income, and debt sum makes sense while for assets and liabilities, max would be more appropriate. It's okay if groups have used other financial instruments. This is not an exhaustive list.

In [ ]:
import pandas as pd

def show_financial_statement(driver, cik):
    query = """
      MATCH (c:Company {cik: $cik})-[:HAS_FINANCIAL_INSTRUMENT]->(f:FinancialInstrument)
      WHERE f.type IN ['SalesRevenueNet', 'NetIncomeLoss', 'DebtCurrent', 'Assets', 'LiabilitiesCurrent']
      RETURN
          SUM(CASE WHEN f.type = 'SalesRevenueNet' THEN f.value ELSE 0 END) AS total_sales_revenue_net,
          SUM(CASE WHEN f.type = 'NetIncomeLoss' THEN f.value ELSE 0 END) AS total_net_income_loss,
          SUM(CASE WHEN f.type = 'DebtCurrent' THEN f.value ELSE 0 END) AS total_debt_current,
          MAX(CASE WHEN f.type = 'Assets' THEN f.value ELSE NULL END) AS max_assets,
          MAX(CASE WHEN f.type = 'LiabilitiesCurrent' THEN f.value ELSE NULL END) AS max_liabilities_current
    """
    with driver.session() as session:
        result = session.run(query, cik=cik)
        return [record.data() for record in result]

results = show_financial_statement(driver, ciks[0])
pd.DataFrame(results)


,total_sales_revenue_net,total_net_income_loss,total_debt_current,max_assets,max_liabilities_current
0,6.436640e+11,-1.337200e+10,1.660800e+10,2.231300e+10,6.482000e+09


Q2: Cluster companies based on financial health. Financial health is a function of revenue and debt, i.e. a health company will have high revenue and low debt.

---
Considering that the data already contains revenue and debt attributes, clustering is just a matter of segmenting the data based on the matrix of HH, HL, LH, and LL.

---

Other solutions include clustering the data directly in the graph or doing the clustering/grouping outside of the graph.


In [ ]:
import pandas as pd

def show_financial_health(driver):
    query = """
      MATCH (c:Company)-[:HAS_FINANCIAL_INSTRUMENT]->(f:FinancialInstrument)
      WHERE f.type IN ['SalesRevenueNet', 'DebtCurrent']
      WITH
          c,
          SUM(CASE WHEN f.type = 'SalesRevenueNet' THEN f.value ELSE 0 END) AS total_sales_revenue_net,
          SUM(CASE WHEN f.type = 'DebtCurrent' THEN f.value ELSE 0 END) AS total_debt_current
      WITH
          c,
          total_sales_revenue_net,
          total_debt_current,
          CASE
              WHEN total_sales_revenue_net > 1000000 AND total_debt_current > 500000 THEN 'High Revenue & High Debt'
              WHEN total_sales_revenue_net > 1000000 AND total_debt_current <= 500000 THEN 'High Revenue & Low Debt'
              WHEN total_sales_revenue_net <= 1000000 AND total_debt_current > 500000 THEN 'Low Revenue & High Debt'
              WHEN total_sales_revenue_net <= 1000000 AND total_debt_current <= 500000 THEN 'Low Revenue & Low Debt'
              ELSE 'Unknown'  // Handle unexpected cases
          END AS financial_health_category
      RETURN c.name AS company_name, financial_health_category
    """
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

results = show_financial_health(driver)
pd.DataFrame(results)


,company_name,financial_health_category
0,LOWES COMPANIES INC,High Revenue & Low Debt
1,WALT DISNEY CO/,High Revenue & Low Debt
2,"MACY'S, INC.",High Revenue & High Debt
3,COMCAST CORP,Low Revenue & High Debt
4,J C PENNEY CO INC,High Revenue & Low Debt
5,XEROX CORP,Low Revenue & High Debt
6,WAL MART STORES INC,High Revenue & Low Debt
7,BOEING CO,Low Revenue & High Debt
8,PFIZER INC,High Revenue & High Debt


Q3: Identify unusual reporting patterns or significant deviations from historical data, which may indicate potential fraud or misrepresentation.

---
Because I loaded all financial instruments for a company across all quarters, detecting fraud is just a function of looking at anamolies. We can use a simple metric for taking the average of each instrument and looking at any instrument that is 1.5 STD DEV above the average. Other answers can use a different formula or comparing the values across similar companies.


In [ ]:
import pandas as pd

def detect_financial_anomalies_per_company(driver):
    query = """
      MATCH (c:Company)-[:HAS_FINANCIAL_INSTRUMENT]->(f:FinancialInstrument)
      WHERE f.value IS NOT NULL
      WITH
          c.name AS company_name,
          f.type AS financial_type,
          f.date AS financial_date,
          f.value AS financial_value,
          AVG(f.value) AS avg_value,
          STDEV(f.value) AS stdev_value
      WITH
          company_name,
          financial_type,
          financial_date,
          financial_value,
          avg_value,
          stdev_value,
          CASE
              WHEN financial_value > avg_value + (1.5 * stdev_value) THEN 'Anomaly'
              ELSE 'Normal'
          END AS status
      WHERE status = 'Anomaly'
      RETURN company_name, financial_type, financial_date, financial_value, avg_value, stdev_value, status
      ORDER BY company_name, financial_type, financial_date
    """
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

results = detect_financial_anomalies_per_company(driver)
pd.DataFrame(results)


""


Q4: Analyze how corporate executives and board members are connected across different
companies and perform centrality analysis. Note that this dataset directly does not
contain the names of office holders for a company. This data will need to be extracted
from other sources such as Form 8-K or Form 10-K, or any third party data source.

---
I'm fetching form 10-K from the SEC website directly and then extracting the names of officers. The names are there in a table so just searching for tables with "Age" and then extracting the names. I'm then converting them into Person nodes to be added to Neo4J and linking against a company. Doing an upsert of the person node. The same data can be fetched from other sources too.


In [ ]:
import requests
from bs4 import BeautifulSoup
import re

def fetch_executive_officers(cik, start_date, end_date, headers):
    url = f"https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={cik}&type=10-K&datea={start_date}&dateb={end_date}&owner=exclude&count=100"

    response = requests.get(url, headers=headers)
    html_content = response.content

    soup = BeautifulSoup(html_content, 'html.parser')

    table = soup.find('table', class_='tableFile2')

    rows = table.find_all('tr')

    for row in rows:
        columns = row.find_all('td')
        if len(columns) > 0:
            filing_type = columns[0].text.strip()
            if filing_type == '10-K':
                document_link = 'https://www.sec.gov' + columns[1].a['href']
                print(f"Found 10-K filing for CIK {cik}: {document_link}")

                doc_response = requests.get(document_link, headers=headers)
                doc_soup = BeautifulSoup(doc_response.content, 'html.parser')

                rows = doc_soup.find_all('tr')
                for row in rows:
                    if row.find('td', string="10-K"):
                        file_link = row.find('a')['href']
                        break

                file_response = requests.get('https://www.sec.gov/' + file_link, headers=headers)
                file_soup = BeautifulSoup(file_response.content, 'html.parser')

                executive_officers = extract_executive_officers(file_soup, cik)

                if executive_officers:
                    return executive_officers

def extract_executive_officers(file_soup, cik):
    tables = file_soup.find_all('table')

    executive_officers = []
    for table in tables:
        rows = table.find_all('tr')

        for row in rows:
            columns = row.find_all(['th', 'td'])
            if any('Age' == col.get_text(strip=True) for col in columns):
                for data_row in rows[1:]:
                    cells = data_row.find_all('td')
                    if len(cells) > 0:
                        name = cells[0].text.strip()
                        if name and name.lower() != 'name':
                            name = re.sub(r'\s+', ' ', name)
                            executive_officers.append(name)
                break
    return executive_officers


start_date = '20090101'
end_date = '20091231'
headers = {
    'User-Agent': 'GWU (zubair.nabi@gwu.edu)'
}

officer_queries = []
for cik in ciks:
    print(f"\nFetching executive officers for CIK {cik}")
    executive_officers = fetch_executive_officers(cik, start_date, end_date, headers)
    if executive_officers:
      for officer in executive_officers:
        query = f"""
        MATCH (c:Company {{cik: '{cik}'}})
        MERGE (p:Person {{name: '{officer}'}})
        MERGE (c)-[:HAS_OFFICER]->(p);
        """
        officer_queries.append(query)
print(officer_queries)


Fetching executive officers for CIK 794367
Found 10-K filing for CIK 794367: https://www.sec.gov/Archives/edgar/data/794367/000119312509070344/0001193125-09-070344-index.htm

Fetching executive officers for CIK 60667
Found 10-K filing for CIK 60667: https://www.sec.gov/Archives/edgar/data/60667/000006066709000036/0000060667-09-000036-index.htm

Fetching executive officers for CIK 1166126
Found 10-K filing for CIK 1166126: https://www.sec.gov/Archives/edgar/data/1166126/000119312509069467/0001193125-09-069467-index.htm

Fetching executive officers for CIK 104169
Found 10-K filing for CIK 104169: https://www.sec.gov/Archives/edgar/data/104169/000010416909000006/0000104169-09-000006-index.htm

Fetching executive officers for CIK 78003
Found 10-K filing for CIK 78003: https://www.sec.gov/Archives/edgar/data/78003/000119312509040568/0001193125-09-040568-index.htm

Fetching executive officers for CIK 1076405
Found 10-K filing for CIK 1076405: https://www.sec.gov/Archives/edgar/data/1076405/

In [ ]:
def insert_officer_data(tx):
    for query in officer_queries:
      tx.run(query)

with driver.session() as session:
    session.execute_write(insert_officer_data)

In [ ]:
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget

g = Neo4jGraphWidget(driver)

def show_graph(driver):
    query = f"""
    MATCH (c:Company {{cik: '{ciks[0]}'}})-[r:HAS_OFFICER]->(p:Person)
    RETURN c, r, p;
    """
    g.show_cypher(query)

show_graph(driver)

GraphWidget(layout=Layout(height='650px', width='100%'))

Q4: To find officers who are a part of multiple companies, just need to perform a query that looks up person nodes that are connected to multiple company nodes. Unfortunately, the 16 companies that I chose do not have any such officers. :( But for the entire dataset there should be some.

In [ ]:
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget

g = Neo4jGraphWidget(driver)

def show_graph(driver):
    query = f"""
    MATCH (p:Person)-[r:HAS_OFFICER]->(c:Company)
    WITH p, COLLECT(c) AS companies
    WHERE SIZE(companies) > 1
    RETURN p, companies;
    """
    g.show_cypher(query)

show_graph(driver)

GraphWidget(layout=Layout(height='500px', width='100%'))

In [ ]:
pip install neo4j-graphrag openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.6/179.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3
  Attempting uninstall: fsspec
    Found exist

In [ ]:
from google.colab import userdata

import os
os.environ["OPENAI_API_KEY"] = userdata.get('openai_key')

Q5: Financial Query and Report Generation via GraphRAG. The user should be able to
query specific financial metrics, trends, or insights from the dataset using plain English

---
I have a paid OpenAI subscription for using GPT4 but any LLM can be used here. In addition, I'm using the official GraphRAG package from Neo4J it's perfectly fine to do prompt engineering directly to run a query on Neo4J. Also, I'm using the Text2CypherRetriever but it's perfectly fine to put embeddings in the graph nodes and then use an embedding based retriever.



In [ ]:
from neo4j_graphrag.retrievers import Text2CypherRetriever
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.generation import GraphRAG

llm = OpenAILLM(model_name="gpt-4o", model_params={"temperature": 0})

retriever = Text2CypherRetriever(
    driver,
    llm
)
rag = GraphRAG(retriever=retriever, llm=llm)

In [ ]:
query_text = "How many companies are there?"
rag.search(query_text=query_text)

RagResultModel(answer='There are 16 companies.', retriever_result=None)

In [ ]:
query_text = "What are the names of the companies?"
rag.search(query_text=query_text)

RagResultModel(answer="The names of the companies are:\n\n1. LOWES COMPANIES INC\n2. WALT DISNEY CO\n3. JOHNSON & JOHNSON\n4. VISA INC.\n5. MACY'S, INC.\n6. COMCAST CORP\n7. J C PENNEY CO INC\n8. XEROX CORP\n9. WAL MART STORES INC\n10. CITIGROUP INC\n11. WELLS FARGO & CO/MN\n12. PEPSI BOTTLING GROUP INC\n13. BOEING CO\n14. AMERICAN EXPRESS CO\n15. J P MORGAN CHASE & CO\n16. PFIZER INC", retriever_result=None)

In [ ]:
query_text = "Which company has 794367 cik?"
rag.search(query_text=query_text)

RagResultModel(answer="The company with CIK 794367 is MACY'S, INC.", retriever_result=None)

In [ ]:
query_text = "Is Amercian Express Co in the data?"
rag.search(query_text=query_text)

RagResultModel(answer='Yes, American Express Co is in the data.', retriever_result=None)

In [ ]:
query_text = "How many companies are in CA?"
rag.search(query_text=query_text)

RagResultModel(answer='There are 3 companies in CA.', retriever_result=None)

In [ ]:
query_text = "Which companies are in CA?"
rag.search(query_text=query_text)

RagResultModel(answer='The companies in CA are WALT DISNEY CO, VISA INC., and WELLS FARGO & CO/MN.', retriever_result=None)

In [ ]:
query_text = "Give me the names of the companies in CA and their cities"
rag.search(query_text=query_text)

RagResultModel(answer='- WALT DISNEY CO/ in Burbank\n- VISA INC. in San Francisco\n- WELLS FARGO & CO/MN in San Francisco', retriever_result=None)

In [ ]:
query_text = "How many financial instrument types are there and what are they?"
rag.search(query_text=query_text)

RagResultModel(answer='There are 5 financial instrument types: NetIncomeLoss, SalesRevenueNet, Assets, LiabilitiesCurrent, and DebtCurrent.', retriever_result=None)

In [ ]:
def clear_graph(tx):
    query = """
    MATCH (n)
    DETACH DELETE n
    """
    tx.run(query)

with driver.session() as session:
    session.execute_write(clear_graph)